In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [3]:
FILE_PATH_CSV_SCORES = './model3_final_scores.csv'

In [4]:
df_scores = pd.read_csv(FILE_PATH_CSV_SCORES,)
df_scores.head()

,Result,Castle_size,Cycles
0,Lose,10,12
1,Lose,11,6
2,Lose,10,38
3,Lose,11,204
4,Lose,10,544
